In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from importlib import reload
import json
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import MultiLearn_GLRM

In [31]:
df_train = pd.read_csv('./cancer/xtrain.data', sep='\s+', header=None)
df_label_train = pd.read_csv('./cancer/xtrain_label.data', sep='\s+', header=None)
df_test = pd.read_csv('./cancer/xtest.data', sep='\s+', header=None)
df_label_test = pd.read_csv('./cancer/xtest_label.data', sep='\s+', header=None)

df_train.loc[-1] = df_label_train.loc[0].to_list()
df_train.index = df_train.index + 1  
df_train = df_train.sort_index()
df_train = df_train.T

df_test.loc[-1] = df_label_test.loc[0].to_list()
df_test.index = df_test.index + 1  
df_test = df_test.sort_index()
df_test = df_test.T


X_train = df_train.drop(columns=[0])
Y_train = df_train[0]
Y_train = np.array(Y_train, dtype=np.int64).reshape(-1,1)

X_test = df_test.drop(columns=[0])
Y_test = df_test[0]
Y_test = np.array(Y_test, dtype=np.int64).reshape(-1,1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


params = {
    "train_hyper_params": None,
    "test_hyper_params": None,
    "beta": None,
    "embedding_matrix": None,
    "train_error": None,
    "test_error": None,
    "embedding_dim": None,
    "predictor_scaling_parmas": None,
    "target_scaling_parmas": None
}

In [32]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=144)
# X_train_pca = pca.fit_transform(X_train)

n_class = max(max(Y_train), max(Y_test))[0]

one_hot_targets = np.eye(n_class)[Y_train.reshape(-1,)-1]
Train = np.hstack((X_train_pca, one_hot_targets))

In [33]:
Train.shape

(144, 158)

In [34]:
params["predictor_scaling_parmas"] =  {
    "mean" : scaler.mean_.tolist(),
    "var" : scaler.mean_.tolist()
}

In [70]:
reload(MultiLearn_GLRM)
from MultiLearn_GLRM import Multi_Learn, generate_AXY
from MultiLearn_GLRM.Params.semisupervised_params import Multilearn_GLRM_Semisupervised_Train_Params, Multilearn_GLRM_Semisupervised_Test_Params

embedding_dim = 10
params["embedding_dim"] = embedding_dim

n_class = n_class

A_prime, X_prime, Y_prime = generate_AXY.get_semisupervised_glrm_train_form(Train[:,:-n_class], Train[:,-n_class:], n_class, embedding_dim)
GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions = Multilearn_GLRM_Semisupervised_Train_Params(A_prime, X_prime, Y_prime, n_class)

params["train_hyper_params"] = [GLRM_loss_list, X_regulariation_list, Y_regulariation_list]

num_iterations=100
learning_rate=0.5
result = Multi_Learn.alternating_minimization(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions, num_iterations, learning_rate, n_class)


epoch: 0
iter: 0 Total loss: tf.Tensor(184.46474863733135, shape=(), dtype=float64) [<tf.Tensor: shape=(), dtype=float64, numpy=13.770082555309555>, <tf.Tensor: shape=(), dtype=float64, numpy=121.0170630334533>]
iter: 5 Total loss: tf.Tensor(167.99011018350282, shape=(), dtype=float64)
iter: 0 Total loss: tf.Tensor(167.59401771604348, shape=(), dtype=float64)
iter: 6 Total loss: tf.Tensor(102.96721050529024, shape=(), dtype=float64)
epoch: 1
iter: 0 Total loss: tf.Tensor(100.79053489402534, shape=(), dtype=float64) [<tf.Tensor: shape=(), dtype=float64, numpy=13.730577190649857>, <tf.Tensor: shape=(), dtype=float64, numpy=94.53722614243927>]
iter: 11 Total loss: tf.Tensor(78.10472769147424, shape=(), dtype=float64)
iter: 0 Total loss: tf.Tensor(77.83287721998386, shape=(), dtype=float64)
iter: 6 Total loss: tf.Tensor(65.99857643690643, shape=(), dtype=float64)
epoch: 2
iter: 0 Total loss: tf.Tensor(64.05095898663163, shape=(), dtype=float64) [<tf.Tensor: shape=(), dtype=float64, numpy=1

In [71]:
beta = result[1][1:,0:n_class]
Y_final = result[1][1:embedding_dim+1,n_class:]
params["beta"] = beta.numpy().tolist()
params["embedding_matrix"] = Y_final.numpy().tolist()

train_prob = tf.nn.softmax(np.matmul(result[0], result[1][:,0:n_class])).numpy()
label_train_shuffled =  np.argmax(Train[:,-n_class:], axis=1)
params["train_accuracy"] = accuracy_score(label_train_shuffled, np.argmax(train_prob, axis=1))
params["train_accuracy"]

0.11805555555555555

In [72]:
label_train_shuffled

array([ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  2,
        2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,
        5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,
        7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,
        9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11,
       11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13])

In [73]:
np.argmax(train_prob, axis=1)

array([ 2,  1,  1,  1,  2,  1,  1,  2,  5,  2,  1,  1,  6,  8,  2,  8,  6,
        2,  2,  1,  1,  6,  2, 13,  6,  2,  8,  1, 11, 11,  2,  2,  6,  4,
        4,  4,  1,  4,  4,  4,  7,  4,  1,  6,  9,  6,  7,  4,  2,  2,  2,
        2,  7,  1,  2,  7,  1,  6,  4,  1,  2,  2,  1,  4,  5,  5,  2,  5,
        6,  5,  3,  4,  5,  6, 11,  4,  5,  4,  8,  8,  4,  5,  4,  5,  5,
        4,  4,  4, 11,  5,  4,  4,  6,  4,  4, 11,  1,  6, 13,  2, 13, 11,
        6,  1,  6,  2,  1,  5,  4,  1,  4,  2,  2,  6,  4,  1,  5, 11,  6,
        8,  2,  2,  4,  5,  2,  5,  2,  2,  6,  1,  6, 12,  2,  5,  6,  5,
        6,  4,  4,  5,  2,  6,  8,  4])

In [ ]:
# res = np.matmul(result[0], result[1][:,0:n_class])
# np.mean((Y_train - res)**2)
# 0.4495674496068231

# res = np.matmul(result[0], result[1][:,0:n_class])
# np.mean((Y_train - res)**2)
# 0.4011543372347976


In [ ]:
A_prime.shape

In [ ]:
"""
<tf.Tensor: shape=(11, 1), dtype=float64, numpy=
array([[-0.02956712],
       [-0.43500185],
       [ 0.46663761],
       [ 0.28479513],
       [-0.04591487],
       [ 0.22732162],
       [ 0.19878154],
       [-0.21300145],
       [ 0.0071286 ],
       [ 0.11102323],
       [-0.00936144]])>
"""

In [ ]:
for i in range(0, Y_train.shape[0]):
    print(Y_train[i], res[i])

In [ ]:
reload(MultiLearn_GLRM)
from MultiLearn_GLRM import Multi_Learn, generate_AXY
from MultiLearn_GLRM.Params.semisupervised_params import Multilearn_GLRM_Semisupervised_Train_Params, Multilearn_GLRM_Semisupervised_Test_Params

A = X_test
A_prime =tf.constant(A)


X = np.random.standard_normal((A.shape[0], embedding_dim))
X_prime = X
X_prime = tf.Variable(X_prime)

Y_prime = Y_final

GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, X_grad_restrictions, Y_grad_restrictions =  Multilearn_GLRM_Semisupervised_Test_Params(A_prime, X_prime, Y_prime)
params["test_hyper_params"] = [GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list]

num_iterations=10000
learning_rate=0.004
results_log = []
for i in range(0,10):
    A = X_test
    A_prime =tf.constant(A)


    X = np.random.uniform(-1,1, (A.shape[0], embedding_dim))
    X_prime = X
    X_prime = tf.Variable(X_prime)

    Y_prime = Y_final

    result_val = Multi_Learn.predict(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, num_iterations, learning_rate)
    val_pred = np.hstack((result_val[0], A_prime))
    val_pred = np.hstack((val_pred, np.ones(shape=(val_pred.shape[0],1))))
    val_pred = np.matmul(val_pred, beta)
    val_pred = tf.nn.softmax(val_pred).numpy()
    accuracy = accuracy_score(np.array(label_test)-1, np.argmax(val_pred, axis=1))

    print(accuracy)
    results_log.append(accuracy)


params["accuracy"] = results_log

# with open('./logs/Prostate_'+ str(round(np.mean(results_log),5))+'_MSE.json', 'w') as f:
#     f.write(json.dumps(params, sort_keys=True, indent=4, separators=(',', ': ')))




In [ ]:
params

In [ ]:
val_pred = np.hstack((result_val[0], A_prime))
val_pred = np.hstack((val_pred, np.ones(shape=(val_pred.shape[0],1))))
regress_val = np.matmul(val_pred, beta)
np.mean((scaler2.inverse_transform(Y_test) - scaler2.inverse_transform(regress_val))**2)

In [ ]:
regress_val

In [ ]:
Y_train

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, predictions_val), confusion_matrix(Y_train, predictions)

In [ ]:
409/(409+200), 409/(409+857)

In [ ]:
305/(305+133), 305/(305+961)

In [ ]:
2027/(2027+538), 2027/(2027+3343)

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train_embedding = result[0][:, 1:embedding_dim+1]
X_train_embedding = X_train_embedding.numpy()

X_test_embedding = result_val[0].numpy()


clf = LogisticRegression().fit(X_train_embedding, Y_train)
pred_logistic = clf.predict(X_train_embedding)
pred_logistic_val = clf.predict(X_test_embedding)
accuracy_score(Y_train, pred_logistic), accuracy_score(Y_test, pred_logistic_val)

In [ ]:
confusion_matrix(Y_test, pred_logistic_val), confusion_matrix(Y_train, pred_logistic)

In [ ]:
328/(328+236), 328/(328+938)

In [ ]:
1565/(1565+650), 1565/(1565+3805)

In [ ]:
beta[5:]

In [ ]:
A_prime[0]

In [ ]:
x_embeddings

In [ ]:
prob_val[0:10]

In [ ]:
prob[0:10]

In [ ]:
temp = tf.Variable([1,1,1])
tf.where(temp!=1).shape[0]
# tf.where([True, False, False, True]).numpy()

In [ ]:
np.min(val_pred,axis=0)

In [ ]:
np.min(result[0],axis=0)

In [ ]:
beta

In [ ]:
result[0][3:10,:]

In [ ]:
'beta' in params.keys()